In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import numpy as np


df = pd.read_csv('soil_data.csv')


X = df[['N', 'P', 'K', 'pH', 'Moisture', 'Temperature']]
y_crop = df['Recommended Crop']
y_fertilizer = df['Recommended Fertilizer & Quantity']
y_yield = df['Predicted Yield']


crop_encoder = OneHotEncoder()
fertilizer_encoder = OneHotEncoder()

y_crop_encoded = crop_encoder.fit_transform(y_crop.values.reshape(-1, 1)).toarray()
y_fertilizer_encoded = fertilizer_encoder.fit_transform(y_fertilizer.values.reshape(-1, 1)).toarray()


y_combined = np.concatenate([y_crop_encoded, y_fertilizer_encoded, y_yield.values.reshape(-1, 1)], axis=1)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['N', 'P', 'K', 'pH', 'Moisture', 'Temperature'])
    ]
)


model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(n_estimators=10, n_jobs=-1)))
])


X_train, X_test, y_train, y_test = train_test_split(X, y_combined, test_size=0.2, random_state=42)


model.fit(X_train, y_train)


joblib.dump(model, 'soil_model.pkl')
joblib.dump(crop_encoder, 'crop_encoder.pkl')
joblib.dump(fertilizer_encoder, 'fertilizer_encoder.pkl')


['fertilizer_encoder.pkl']